## Shortcourse Notebook

We will take breaks from the powerpoint presentations to apply concepts learned in the course. Follow along as we go through these examples together, and feel free to take your own notes in this Jupyter Notebook for future reference. 

You can create new markdown cells (the cells like this one, with text but no python code) or double-click on the existing markdown cells to add text. Alternatively, you can put comments in your code cells, which are preceded by a ```#``` symbol. 

In [ ]:
# Import necessary modules 
from earthscopestraintools.mseed_tools import ts_from_mseed
from earthscopestraintools.gtsm_metadata import GtsmMetadata
from earthscopestraintools.timeseries import plot_timeseries_comparison

# Establish logging session
import logging
logger = logging.getLogger()
logging.basicConfig(
        format="%(message)s", level=logging.INFO
    )


## Aquire data and metadata

**Exercise:** Gather metadata and 1 Hz raw data from B073 from May 5th, 2012 to May 10th, 2012 by filling in the blanks. 

Metadata is acquired using the ```GTSMMetadata()``` function of the gtsm_metadata module, which we imported in the previous cell. Data can be acquired through the ```ts_from_mseed()``` function in the mseed_tools module. The [API documentation](https://earthscopestraintools.readthedocs.io/en/latest/api/earthscopestraintools.html#submodules) describes the function inputs in further detail. 

More detailed description the seed codes to acquire the correct data are listed [here](https://earthscopestraintools.readthedocs.io/en/latest/pages/DataAvailability.html). And stucture of the Metadata and Timeseries classes that will hold information for our station and data are in the introductory material Metadata and Data Structures tabs in the [docs](https://earthscopestraintools.readthedocs.io/en/latest).


In [ ]:
# Get metadata and data

# First, set parameters for the seed code
network = 
station = 
location = 
sample_rate = 

start = 
end = 

# Get the metadata
meta = 

# Get the mseed data from the IRIS DMC using earthscopestraintools
strain_raw = 

In [ ]:
# Examine the metadata contents through the .show() call 
meta

In [ ]:
# Run this code cell to look at available attributes and functions of strain_raw
print('Attributes:')
funcs = []
print(list(vars(strain_raw).keys()))
for s in dir(strain_raw): 
    if s.startswith('_') != True and s not in vars(strain_raw).keys(): funcs.append(s)
print('Functions')
print(funcs)


In [ ]:
# Preview the data of strain_raw with the data attribute
strain_raw.

In [ ]:
# Use one of the functions to plot the data
strain_raw.

## Filter and decimate to 5 minutes

There appears to be an interesting signal on the 8th of May... but 1 sps data is overkill for analyzing the signal. To decrease later computation time, let's filter and decimate the data to a 5 minute sample period. 

**Exercise:** Apply the filtering function designed according to specifications in [Agnew and Hodgkinson, 2008](https://pubs.geoscienceworld.org/ssa/bssa/article/97/1B/91/146607/Designing-Compact-Causal-Digital-Filters-for-Low?casa_token=_GVDmXKT-RAAAAAA:PPJHivgGPh2x1dbRv0BYgio-vRkUMX44unECYJoHH6tYCmloKOkGloE0paUrkmEWaD6avQ) with the ```decimate_1s_to_300s()``` function in the timeseries module.

In [ ]:
# Filter and decimate strain_raw
strain_raw5min = 

## Linearization

**Exercise:** Convert the raw data in counts to microstrain through the ```linearize()``` function of the timeseries module. Find this function in the docs, and note that reference strains can be found in the metadata. 

In [ ]:
# Linearize
gauge_microstrain = 
# Preview the data
gauge_microstrain.data

## Calculate regional strains

**Exercise:** Examine and apply the available calibration matrices to produce regional areal, differential, and engineering shear strains. 

In [ ]:
# Which strain matrices are available in the metadata?
# the calibration matrices are found in a dictionary object 
# within the metadata called with the .strain_matrices attribute
meta.

In [ ]:
# Apply both strain matrices to the gauge microstrain
lab_strain = 
tide_strain = 

In [ ]:
# Use the plot_timeseries_comparison() function to plot 
# a comparison of regional strains
plot_timeseries_comparison()

## Signal correction

**Exercise:** Correct the gauge strain timeseries for barometric pressure, tides, trend and optionally offsets. 


### Barometric Pressure Correction

Seed codes for applying the pressure correction are listed on this [page](https://earthscopestraintools.readthedocs.io/en/latest/pages/DataAvailability.html).

In [ ]:
# First, get the atmospheric pressure data
atmp = 

In [ ]:
# Plot the correction
atmp.

In [ ]:
# Examine the pressure response coefficients per gauge
# in microstrain/hPa
meta.

In [ ]:
# Interpolate the pressure data to the gauge strain timestamps using the 
# interpolate() function
atmp_interp = 

In [ ]:
# Calculate the pressure correction using the 
# calculate_pressure_correction() function with atmp_response
# coefficients in the metadata
atmp_c = 

In [ ]:
# Plot the uncorrected and pressure corrected timeseries
# The .apply_corrections() function is useful for correcting the gauge strain


### Tide Correction

The tidal correction will run SPOTL (Some Programs for Ocean Tide Loading; Agnew, 2012), a fortran program in the docker container.


In [ ]:
# Take a look at the metadata for the tides by calling
# tidal_params
meta.

In [ ]:
# Calculate tidal corrections for each gauge using the
# calculate_tide_correction function with the tidal_params and station longitude
tide_c = 

### Trend Correction

The [API documentation](https://earthscopestraintools.readthedocs.io/en/latest/api/earthscopestraintools.processing.html#earthscopestraintools.processing.calculate_linear_trend_correction) lists options for computing the linear trend.

In [ ]:
# Calculate the trend correction for each gauge
# Sometimes correcting for pressure first is best, 
# because the pressure can have its own trend
trend_c = 

In [ ]:
# Plot various applied corrections to see the effect on the final corrected strain
p1 = 
p2 = 
gauge_strain_corr = 

plot_timeseries_comparison([gauge_microstrain,p1,p2,gauge_strain_corr],zero=True,
                           names=['Original','+Trend Corrected','+Pressure Corrected','+Tide Corrected'])

### Offset correction

See how the simple offset correction calculation is implemented in the [docs](https://earthscopestraintools.readthedocs.io/en/latest/api/earthscopestraintools.processing.html#earthscopestraintools.processing.calculate_offsets).

In [ ]:
# Calculate offsets via first differencing above a cutoff limit 
# adjusted to the noise of the data

offset_c = 

In [ ]:
# Plot the offset corrections
# try adjusting the offset calculation parameters in the previous cell to see their effect


**Whether you choose to apply these offsets is up to you, depending on your purpose forusing the data.**

## Tectonic or Nontectonic?

This can be hard to say the origin of a signal sometimes, but one common environmental factor that causes seemingly anomolous strains is rainfall. Let's get the data and examine the timing of rainfall relative to the change in the strain time series. 


In [ ]:
# First, let's transform the corrected gauge strain to regional strain
reg_strain_corr = 

In [ ]:
# Get the rainfall data
rain = 

In [ ]:
# Plot the corrected regional strains with rainfall 
# and atmospheric pressure 


## Strain axes

Let's look at the strain axes alignment in a gif animation. Take a look at the code implementation [here](https://earthscopestraintools.readthedocs.io/en/latest/api/earthscopestraintools.timeseries.html#earthscopestraintools.timeseries.Timeseries.strain_video).

In [ ]:
# Strain gif using strain_video() function
%matplotlib widget
